In [4]:
import requests
import tensorflow as tf
import json
import base64

def prepare_example(review_text):
    # Define the features of your data
    feature = {
        "Review": tf.train.Feature(bytes_list=tf.train.BytesList(value=[review_text.encode()]))
    }

    # Create an Example
    example = tf.train.Example(features=tf.train.Features(feature=feature))

    # Serialize the Example to a string
    example_str = example.SerializeToString()

    return example_str

def get_prediction(example_str, endpoint):
    # Encode Example string to base64
    encoded_example_str = base64.b64encode(example_str).decode()

    # Prepare the JSON data
    json_data = {
        "signature_name": "serving_default",
        "instances": [{"b64": encoded_example_str}]
    }

    # Send POST request
    response = requests.post(endpoint, json=json_data)

    # Extract predictions from response
    prediction = response.json().get("predictions")

    # Determine result based on prediction
    if prediction:
        prediction_value = prediction[0][0]
        result = "Good" if prediction_value > 0.6 else "Bad"
    else:
        result = "Error: No predictions found in response."

    return result

def main():
    review_text = input("Enter your review: ")
    endpoint = "http://103.190.215.173:8501/v1/models/food-review:predict"

    # Prepare example
    example_str = prepare_example(review_text)

    # Get prediction result
    result = get_prediction(example_str, endpoint)

    print(result)

if __name__ == "__main__":
    main()


Good
